In [9]:
from __future__ import division
import numpy as np
import hashlib
import ipdb
import cPickle as pickle
import itertools
assert hashlib.md5("a").hexdigest() == hashlib.md5("a").hexdigest()

from pi import make_pi

# http://nlp.stanford.edu/IR-book/html/htmledition/near-duplicates-and-shingling-1.html

In [10]:
# shingle is 542
docs = []
docs.append("0 2 9 0 5 4 3 4 5 1 5 4 9".split())
docs.append("1 2 9 0 5 4 3 4 5 1 5 4 9".split())
docs.append("5 4 2 4 5 9 1 9 9 3 4 2 4".split())
docs.append("5 5 5 4 2 7 3 9 1 4 5 4 9".split())
docs.append("4 4 4 4 5 6 3 7 5 4 3 3 9".split())
docs.append("1 4 3 2 5 7 3 7 5 7 5 7 9".split())
docs.append("12 42 23 252 25 27 23 27 25 17 25 37 49".split())

def shingle(doc, n):
    shingles = zip(*[doc[i:] for i in range(n)])
    return [" ".join(a) for a in shingles]


In [14]:
HASH_LEN = 32
ngram = 4
iters = 100

def get_pi():
    with open("pi.p", "r") as outf:
        return pickle.load(outf)

def permute(h, pi):
    '''permute h and return a hex string'''
    assert len(h) == len(pi)
    permute = ""
    for c in range(HASH_LEN):
        permute += h[np.where(pi == c)[0][0]]
    return "0x" + permute

def shingle_hash_permute_min(j):
    '''shingle doc j, hash shingles, permute the hashes'''
    pi = get_pi()
    shingles = shingle(docs[j], ngram) # shingle
    hdj = [hashlib.md5(s).hexdigest() for s in shingles] # hash
    pi_d_j = [permute(h, pi) for h in hdj]
    return int(min(pi_d_j), 16)

def jaccard(a, b):
    '''a and b are indexes on documents'''
    return len(set(a).intersection(set(b)))/len(set(a).union(set(b)))

def sanity_check(d1, d2):
    '''as iters grows, this should be more and more like jaccard'''
    equal = 0
    for i in range(iters):
        make_pi(HASH_LEN)
        pi_d = [shingle_hash_permute_min(j) for j in range(len(docs))]
        if pi_d[d1] == pi_d[d2]:
            equal += 1
    # These should be more or less the same, if this is working properly
    # print "fancy jaccard={}\npoor man's jaccard {}".format(equal/iters, jaccard(set(shingle(docs[d1], ngram)), set(shingle(docs[d2], ngram))))
    return equal/iters, jaccard(set(shingle(docs[d1], ngram)), set(shingle(docs[d2], ngram)))
    
def sketch_docs():
    '''get N (iters) sketches of docs'''
    out = []
    for i in range(iters):
        make_pi(HASH_LEN)
        # pi = np.random.permutation(HASH_LEN) # assume for now docs are same size
        out.append([(shingle_hash_permute_min(j), j) for j in range(len(docs))])
    return out


In [12]:
from collections import defaultdict
sketches = sketch_docs()
overlap_check = defaultdict(list)
for sketch in sketches:
    for pairing in sketch:
        hash_no, docno = pairing
        overlap_check[hash_no].append(docno)

candidates = [tuple(value) for (key, value) in overlap_check.items() if len(value) > 1]
candidates = set(candidates)
sketch_jacard = {}
for c in candidates:
    a, b = c
    counts = 0
    for s in sketches:
        # print s[a], s[b]
        if s[a][0] == s[b][0]:
            counts += 1
    sketch_jacard[c] = counts/len(sketches)

print sketch_jacard, jaccard(set(shingle(docs[0], ngram)), set(shingle(docs[1], ngram)))

{(0, 1): 0.78} 0.818181818182


In [13]:
'''another check: avg jaccard diff for all docs'''
count = 0
sum_ = 0
for i in range(6):
    for j in range(6):
        fancy_jac, regular_jac = sanity_check(i,j)
        sum_ += abs(fancy_jac - regular_jac)
        count += 1
        
print sum_/count

0.000833333333333
